<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/4/47/Acronimo_y_nombre_uc3m.png"/>

<img src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" width=15%/>
</center> 

# T5

Excepto el transformer original descrito en el paper "Attention is all you need", el resto de modelos que hemos estudiado únicamente se componen de encoders (BERT)  o decoders (GPT). 

[T5](#https://dl.acm.org/doi/abs/10.5555/3455716.3455856)[1] es un modelo transformador con una arquitectura Seq2Seq, es decir, compuesta por un encoder y un decoder. Precisamente como T5 está basado en una arquitectura Seq2Seq, sigue un enfoque text-to-text, es decir, va a recibir un texto, y va a generar un nuevo texto como salida para cualquier tarea de NLP, aunque dicha el objetivo de dicha tarea no sea generación texto (por ejemplo, clasificación de textos, multietiquetado o NER). 

El método utilizado para entrenar el modelo T5 es muy similar al utilizado por BERT (enmascaramiento de tokens), pero en T5 no solo se enmascaran tokens sino también secuencias de tokens contiguos. Así la predición es una secuencia de tokens que contiene las respuestas (representada por "sentinel tokens"). 

T5 se entrena usando **teacher forcing**, es decir, el modelo siempre va a recibir la secuencia de entrada (representada con los input_ids) seguida de la secuencia de salida u objetivo (la que debe aprender). Las dos secuencias se separan por un token de secuencia de inicio. La secuencia de salida se alimenta al decodificador utilizando decoder_input_ids.

<img src='https://drive.google.com/uc?id=1iPG7UxZPvy6c2iwixQXYetpTzcHiKDuW' width='50%'>


T5 fue entrenado con el corpus “Colossal Clean Crawled Corpus”, que contiene unos ~750GB of de textos en inglés y extraídos de la red (limpiados de etiquetas html). Este dataset es mucho más grande que el usado para entrenar BERT (solo 13 GB de textos), o los 126GBd de textos usados para entrenar XLNet.

Posimplemente por el gran tamaño del corpus usado para pre-entrenar T5, este transformer ha conseguido resultados excelentes en muchas tareas de NLP, superando los obtenidos por enfoques previos [1]. T5 es efectiva tanto para tareas de generación de texto como para tareas de clasificación. 

<img src='https://drive.google.com/uc?id=15XQ-H7IdT3DVtbL7fgwYIm08OYWbB8VZ' width='75%'>

Cuando T5 es ajustado a alguna tarea concreta, en la entrada hay que específica el nombre de la tarea, además de la secuencia de entrada. Por ejemplo, si nuestro objetivo es traducir una oración, será necesario indicar la tarea "translate" y los idiomas de origen y destino "from English to French". 
Si es una tarea de sentiment analysis (binaria), indicarás "cola sentence". 


[1] Raffel, C., Shazeer, N., Roberts, A., Lee, K., Narang, S., Matena, M., ... & Liu, P. J. (2020). Exploring the limits of transfer learning with a unified text-to-text transformer. The Journal of Machine Learning Research, 21(1), 5485-5551.



## Resumen:
- T5 es un transformer basado en Seq2Seq (encoder-decoder), y considera todos las tareas de NLP como una tarea de texto a texto. 
- T5 obtiene buenos resultados tanto para tareas de generación de textos como de clasificación. 
- Es necesario pasar un préfijo para cada tarea. Por ejemplo, para generación de resúmenes, usaremos "summarize". 

- En el encoder, el padding puede hacerse tanto a la izquierda como a la derecha. Como en BERT, las posiciones son absolutas (con respecto al primer token) el padding debe realizarse preferiblemente a la derecha. GPT fue entrenado sin padding. 

- T5 es distribuido en varias versiones: t5-small, t5-base, t5-large, t5-3b, t5-11b.
- Los modelos T5 necesitan una tasa de aprendizaje (learning rate) ligeramente más alta que la predeterminada establecida en el Trainer cuando se usa el optimizador AdamW. Por lo general, 1e-4 y 3e-4 funcionan bien para la mayoría de los problemas (clasificación, resumen, traducción, respuesta a preguntas, generación de preguntas). T5 fue pre-entrenado con el optimizador AdaFactor.

## ¿Cómo utilizar T5? 

Vamos a ver cómo podemos usar T5 para resolver algunas tareas básicas de PLN, como:
- 1) Predecir tokens. 
- 2) Traducción automática. 

T5 necesita un tokenizador especial, por tanto, necesitamos instalar la librería **sentencepiece**:

In [1]:
!pip install transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 5.8 MB/s eta 0:00:00


### T5 para la tarea de modelado de lenguaje basado en enmascaramiento de tokens. 

Para esta tarea, usaremos la clase predefinida **T5ForConditionalGeneration**. 
Para indicar qué huecos debe predecir el modelo, usaremos etiquetas del tipo $<extra\_id\_0>$, $<extra\_id\_1>$, etc. (tantas como huecos).

Por ejemplo, dada la oración "The cute dog walks in the park" una posible secuencia de entrada enmascarada podría ser: 
"The $<extra\_id\_0>$ walks in $<extra\_id_1>$ park".
Vamos a usar el modelo para ver qué tokens predice para esos huecos:

In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

input_ids = tokenizer("The <extra_id_0> walks in <extra_id_1> park.", return_tensors="pt").input_ids
# usamos el método generate para que produzca la salida
# pasamos de ids a palabras
sequence_ids = model.generate(input_ids)
sequences = tokenizer.batch_decode(sequence_ids)
sequences

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['<pad><extra_id_0> park offers<extra_id_1> the<extra_id_2> park.</s>']

Podemos ver que la secuencia de salida generada ha sido '$<pad><extra\_id\_0>$ park offers$<extra\_id\_1>$ the<extra_id_2> park.</s>
Es decir: 
- **park offers"
- ** the**
El decoder ha añadido $<pad>$ como inicio de secuencia, y $</s>$ fin de secuencia. 


No parece que el modelo haya producido una buena salida. 

Vamos a medir el error repecto a una posible salida correcta, como "$<extra\_id\_0>$ cute dog $<extra\_id\_1>$ the $<extra\_id\_2>$". 

In [4]:
input_ids = tokenizer("The <extra_id_0> walks in <extra_id_1> park", return_tensors="pt").input_ids
labels = tokenizer("<extra_id_0> cute dog <extra_id_1> the <extra_id_2>", return_tensors="pt").input_ids

loss = model(input_ids=input_ids, labels=labels).loss
loss.item()

3.783731698989868

Vamos a ver cómo trabajar para otros ejemplos: 

In [5]:
input_ids = tokenizer("The boys <extra_id_0> to <extra_id_1> garden.", return_tensors="pt").input_ids

sequence_ids = model.generate(input_ids)
sequences = tokenizer.batch_decode(sequence_ids)
sequences

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['<pad> The boys are going to the garden.</s>']

In [9]:
input_ids = tokenizer("The government <extra_id_0> to <extra_id_1> the law.", return_tensors="pt").input_ids

sequence_ids = model.generate(input_ids)
sequences = tokenizer.batch_decode(sequence_ids)
sequences

['<pad><extra_id_0> has a right<extra_id_1> enforce<extra_id_2>.</s>']

In [18]:
input_ids = tokenizer("Women <extra_id_0> less <extra_id_1>", return_tensors="pt").input_ids

sequence_ids = model.generate(input_ids)
sequences = tokenizer.batch_decode(sequence_ids)
sequences

['<pad><extra_id_0> are<extra_id_1> likely to be able to survive.</s>']

### T5 para traducción automática

En este caso, tenenos que añadir el prefijo "translate X to Y", donde X e Y son idiomas. El prefijo es necesario porque fue usado durante el pre-entrenamiento de T5. 


In [24]:
input_ids = tokenizer("translate English to French: The house is wonderful.", return_tensors="pt").input_ids
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

input_ids = tokenizer("translate English to German: The house is wonderful.", return_tensors="pt").input_ids
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

input_ids = tokenizer("translate English to Spanish: The house is wonderful.", return_tensors="pt").input_ids
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


La maison est merveilleuse.
Das Haus ist wunderbar.
Das Haus ist wunderbar.


En el ejemplo anterior, solo procesamos un ejemplo. Sin embargo, para entrenar un modelo, debemos usar una colección de ejemplos.

En la tokenización, debemos indicar la longitud máxima para la secuencia de entrada y la longitud máxima para la secuencia de destino.

Además, la función de pérdida debe ignorar los tokens de relleno, por lo que reemplazamos su id con -100, que es el ignore_index de CrossEntropyLoss.


In [28]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# the following 2 hyperparameters are task-specific
max_source_length = 512
max_target_length = 128

# Suppose we have the following 2 training examples:
task_prefix = "translate English to Spanish: "
input_sequence_1 = "Welcome to New York"
input_sequence_2 = "HuggingFace is a company"
input_sequences = [input_sequence_1, input_sequence_2]

output_sequence_1 = "Bienvenidos a Nueva York"
output_sequence_2 = "HuggingFace is una empresa"

encoding = tokenizer(
    [task_prefix + sequence for sequence in input_sequences],
    padding="longest",
    max_length=max_source_length,
    truncation=True,
    return_tensors="pt",
)

input_ids, attention_mask = encoding.input_ids, encoding.attention_mask

# encode the targets
target_encoding = tokenizer(
    [output_sequence_1, output_sequence_2],
    padding="longest",
    max_length=max_target_length,
    truncation=True,
    return_tensors="pt",
)
labels = target_encoding.input_ids

# replace padding token id's of the labels by -100 so it's ignored by the loss
labels[labels == tokenizer.pad_token_id] = -100

# forward pass
loss = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels).loss
loss.item()

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


2.805405855178833